In [1]:
import pandas as pd
import numpy as np

In [3]:
H=['C','M','F','I','L','V','W','Y']
C=['N','Q','D','E','R','K','H']
E=['A','G','T','P','S']

In [4]:
def cvrt(S):
    for i in range(len(S)):
        for j in range(len(H)):
            if S[i] == H[j]:
                S[i]='U'
        for j in range(len(C)):
            if S[i] == C[j]:
                S[i]='X'
        for j in range(len(E)):
            if S[i] == E[j]:
                S[i]='Z'
                
    for i in range(len(S)):
        if S[i] == 'U':
            S[i] = 'H'
        elif S[i] == 'X':
            S[i] = 'C'
        elif S[i] == 'Z':
            S[i] = 'E'
    return S

In [2]:
data = pd.read_csv("PDB_19_Primary.csv")
df=pd.DataFrame(data)
Sequence = df['Sequence'].values.tolist()
print (df)

                                 Pdbid  \
0              >1MBA_Aplysia_limacina1   
1                  >1RCB_Homo_sapiens1   
2            >2HMQ_Themiste_dyscritum1   
3                  >1FHA_Homo_sapiens1   
4              >1RCI_Rana_catesbeiana1   
5                 >1PLC_Populus_nigra2   
6                  >4FGF_Homo_sapiens2   
7   >1NOAStreptomyces_carzinostaticus2   
8             >1CDC_Rattus_norvegicus2   
9                  >1CD8_Homo_sapiens2   
10           >1SHA_Rous_sarcoma_virus3   
11                 >1UBQ_Homo_sapiens3   
12                 >1AYD_Mus_musculus3   
13                   >7RSA_Bos_taurus3   
14                 >3IL8_Homo_sapiens3   
15       >1BKS_Salmonella_typhimurium4   
16                   >2PGD_Ovis_aries4   
17             >2TRX_Escherichia_coli4   
18                 >5P21_Homo_sapiens4   

                                             Sequence  
0   XSLSAAEADLAGKSWAPVFANKNANGLDFLVALFEKFPDSANFFAD...  
1   HKCDITLQEIIKTLNSLTEQKTLCTELTVTDIFAASKNTTEKE

In [5]:
''.join(cvrt(list(Sequence[5])))

'HCHHHEECCEEHEHHEECHEHEEECCHHHCCCEEHECCHHHCCCEHEEEHCEECHEHECCCHHCECECEHCHEHECCECHEHHHEECCEEEHHECHEHC'

In [8]:
Sequence[5]

'HCHHHEECCEEHEHHEECHEHEEECCHHHCCCEEHECCHHHCCCEHEEEHCEECHEHECCCHHCECECEHCHEHECCECHEHHHEECCEEEHHECHEHC'

In [7]:
for i in range(len(Sequence)):
        Sequence[i]=''.join(cvrt(list(Sequence[i])))

In [9]:
def vect(Sequence):
    H = [1,1]
    C = [1,-1]
    E = [-1,1]
    xi =[]
    yi = []
    mi = []
    mixi =[]
    miyi = []
    
    n=len(Sequence)
    i=0
    for ch in Sequence:
    #Calculation of xi,yi
      if ch=='H':
        if(i==0):
          xi.insert(i, H[0])
          yi.insert(i, H[1])
        else:
          xi.insert(i, H[0]+xi[i-1])
          yi.insert(i, H[1]+yi[i-1])
            
      elif ch=='C':
        if(i==0):
          xi.insert(i, C[0])
          yi.insert(i, C[1])
        else:
          xi.insert(i, C[0]+xi[i-1])
          yi.insert(i, C[1]+yi[i-1])
            
      elif (ch=='E'):
        if(i==0):
          xi.insert(i, E[0])
          yi.insert(i, E[1])
        else:
          xi.insert(i, E[0]+xi[i-1])
          yi.insert(i, E[1]+yi[i-1])
            
      #calculation of mi
      mi.insert(i, 1)
      if i>0:
          for j in range(i-1, -1, -1):
              if xi[i]==xi[j] and yi[i]==yi[j]:
                  mi[i] = mi[j]+1
                  break
              
      #calculation of mixi
      mixi.insert(i, mi[i]*xi[i])
        
      #calculation of miyi
      miyi.insert(i, mi[i]*yi[i])
      i=i+1
        
    #calculation of ux, uy, uz
    ux = sum(mixi)/n
    uy = sum(miyi)/n

    # ixx=iyy=izz=ixy=ixz=iyz=0
    ixx=iyy=ixy=0

    for i in range(n):
      ixx=ixx+mi[i]*(xi[i]**2)
      # ixx=ixx+mi[i]*(yi[i]**2+zi[i]**2)
    
    for i in range(n):
      iyy=iyy+mi[i]*(yi[i]**2)
      # iyy=iyy+mi[i]*(xi[i]**2+zi[i]**2)
    
    # for i in range(n):
    #   izz=izz+mi[i]*(zi[i]**2)
    #   # izz=izz+mi[i]*(yi[i]**2+xi[i]**2)
    
    for i in range(n):
      ixy=ixy+mi[i]*xi[i]*yi[i]
    ixy = ixy * (-1)
    
    # for i in range(n):
    #   ixz=ixz+mi[i]*xi[i]*zi[i]
    # ixz = ixz * (-1)
    
    # for i in range(n):
    #   iyz=iyz+mi[i]*yi[i]*zi[i]
    # iyz = iyz * (-1)

    matrix = np.array([[ixx, ixy],
                       [ixy, iyy]])
    # Calculate eigenvalues
    I1, I2 = np.linalg.eigvals(matrix)

    #final vector calculation
    v = [ux/I1, uy/I1, ux/I2, uy/I2]
    return v

In [11]:
vec = []
dmat_mh = []
dmat_ed = []
dmat_sd = []
dmat_mx = []

for k in range(len(Sequence)):
   vec.insert(k,vect(Sequence[k]))

vector=pd.DataFrame(vec)
vector

,0,1,2,3
0,0.004622,0.008056,0.000079,0.000137
1,0.000175,0.000077,0.018709,0.008228
2,0.000192,0.000097,0.008026,0.004055
3,0.000073,0.000015,0.012467,0.002593
4,0.000094,0.000020,0.011696,0.002496
5,0.025663,0.036928,0.000183,0.000264
6,0.000119,0.000092,0.006486,0.005002
7,-0.005436,0.024491,-0.000039,0.000174
8,0.000265,0.000181,0.024664,0.016838
9,0.011668,0.018486,0.000128,0.000202


In [12]:
## Custom function to return Manhattan and Euclidean distance between two vector

# Calculate pairwise Manhattan distance between rows
def pairwise_manhattan_distance(df):
    n = df.shape[0]
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distance_matrix[i, j] = np.sum(np.abs(df.iloc[i] - df.iloc[j]))
            # distance_matrix[j, i] = distance_matrix[i, j]  # Distance matrix is symmetric
    return distance_matrix

# Calculate pairwise Euclidean distance between rows
def pairwise_euclidean_distance(df):
    n = df.shape[0]
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distance_matrix[i, j] = np.sqrt(np.sum((df.iloc[i]-df.iloc[j])**2))
            # distance_matrix[j, i] = distance_matrix[i, j]  # Distance matrix is symmetric
    return distance_matrix

# Calculate pairwise standard distance between rows
def pairwise_standard_distance(df):
    n = df.shape[0]
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distance_matrix[i, j] = np.sum(np.abs(df.iloc[i]-df.iloc[j]))/(1+(np.sum(np.abs(df.iloc[i]-df.iloc[j]))))
            # distance_matrix[j, i] = distance_matrix[i, j]  # Distance matrix is symmetric
    return distance_matrix

def pairwise_maximum_distance(df):
    n = df.shape[0]
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distance_matrix[i, j] = max(np.abs(df.iloc[i]-df.iloc[j]))
            # distance_matrix[j, i] = distance_matrix[i, j]  # Distance matrix is symmetric
    return distance_matrix

In [13]:
dmat_mh = pd.DataFrame(pairwise_manhattan_distance(vector))
dmat_ed = pd.DataFrame(pairwise_euclidean_distance(vector))
dmat_sd = pd.DataFrame(pairwise_standard_distance(vector))
dmat_mx = pd.DataFrame(pairwise_maximum_distance(vector))

In [14]:
dmat_mh['Pdbid']=df['Pdbid']
dmat_mh.set_index('Pdbid', inplace = True)
dmat_mh

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Pdbid,,,,,,,,,,,,,,,,,,,
>1MBA_Aplysia_limacina1,0.0,0.039148,0.024254,0.027434,0.026541,0.050144,0.023739,0.026648,0.053518,0.017590,0.038878,0.044948,0.037090,0.039079,0.073129,0.004521,0.020472,0.046181,0.028882
>1RCB_Homo_sapiens1,0.0,0.000000,0.014893,0.012040,0.012883,0.088829,0.015520,0.056826,0.014760,0.056510,0.001166,0.006791,0.008789,0.007323,0.034797,0.034942,0.019143,0.007609,0.010495
>2HMQ_Themiste_dyscritum1,0.0,0.000000,0.000000,0.006103,0.005405,0.073935,0.002566,0.041967,0.029578,0.041616,0.014866,0.021610,0.013195,0.014947,0.049615,0.020048,0.004325,0.022051,0.004627
>1FHA_Homo_sapiens1,0.0,0.000000,0.000000,0.000000,0.000893,0.077115,0.008513,0.044910,0.026800,0.044796,0.012088,0.018831,0.010416,0.012168,0.046836,0.023228,0.009126,0.019267,0.005637
>1RCI_Rana_catesbeiana1,0.0,0.000000,0.000000,0.000000,0.000000,0.076223,0.007815,0.044057,0.027643,0.043903,0.012931,0.019674,0.011259,0.013011,0.047679,0.022335,0.008477,0.020110,0.004953
>1PLC_Populus_nigra2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.073420,0.043849,0.103199,0.032554,0.088560,0.094630,0.086771,0.088761,0.122810,0.054665,0.070153,0.095863,0.078563
>4FGF_Homo_sapiens2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041306,0.030250,0.041101,0.015538,0.022281,0.013866,0.015618,0.050286,0.019533,0.003654,0.022717,0.005143
>1NOAStreptomyces_carzinostaticus2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071377,0.023303,0.056762,0.063266,0.054923,0.056801,0.091395,0.029278,0.037820,0.063889,0.046383
>1CDC_Rattus_norvegicus2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070880,0.014736,0.008569,0.016454,0.014631,0.020036,0.049312,0.033903,0.007533,0.025255


In [15]:
dmat_ed['Pdbid']=df['Pdbid']
dmat_ed.set_index('Pdbid', inplace = True)
dmat_ed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Pdbid,,,,,,,,,,,,,,,,,,,
>1MBA_Aplysia_limacina1,0.0,0.022271,0.012708,0.015648,0.015021,0.035726,0.012183,0.019269,0.031054,0.012587,0.021935,0.026105,0.019799,0.021103,0.047763,0.003585,0.010814,0.025682,0.015086
>1RCB_Homo_sapiens1,0.0,0.000000,0.011469,0.008410,0.009058,0.049135,0.012642,0.032309,0.010470,0.029676,0.000726,0.004471,0.006202,0.005109,0.026629,0.021194,0.015017,0.007324,0.008549
>2HMQ_Themiste_dyscritum1,0.0,0.000000,0.000000,0.004678,0.003989,0.045620,0.001810,0.026586,0.020982,0.023390,0.011129,0.015711,0.009319,0.010538,0.038097,0.010623,0.003637,0.015563,0.003174
>1FHA_Homo_sapiens1,0.0,0.000000,0.000000,0.000000,0.000777,0.046623,0.006449,0.028136,0.018755,0.025171,0.008328,0.012874,0.009064,0.009635,0.034607,0.013985,0.008108,0.014301,0.004091
>1RCI_Rana_catesbeiana1,0.0,0.000000,0.000000,0.000000,0.000000,0.046406,0.005783,0.027794,0.019337,0.024781,0.008938,0.013516,0.009279,0.009965,0.035343,0.013279,0.007358,0.014729,0.003857
>1PLC_Populus_nigra2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.045514,0.033495,0.053566,0.023151,0.048907,0.050741,0.047950,0.048560,0.064599,0.039079,0.045280,0.050702,0.046404
>4FGF_Homo_sapiens2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026306,0.021693,0.023134,0.012230,0.016720,0.009629,0.011017,0.039221,0.009982,0.002496,0.016001,0.004095
>1NOAStreptomyces_carzinostaticus2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038876,0.018128,0.032079,0.035079,0.030583,0.031464,0.053258,0.021914,0.025705,0.034705,0.027798
>1CDC_Rattus_norvegicus2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036659,0.010443,0.006208,0.012332,0.010774,0.018934,0.030345,0.024185,0.006280,0.017817


In [16]:
dmat_sd['Pdbid']=df['Pdbid']
dmat_sd.set_index('Pdbid', inplace = True)
dmat_sd

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Pdbid,,,,,,,,,,,,,,,,,,,
>1MBA_Aplysia_limacina1,0.0,0.037673,0.023680,0.026701,0.025855,0.047750,0.023188,0.025956,0.050799,0.017286,0.037424,0.043015,0.035763,0.037610,0.068146,0.004501,0.020061,0.044143,0.028071
>1RCB_Homo_sapiens1,0.0,0.000000,0.014675,0.011896,0.012719,0.081582,0.015283,0.053770,0.014545,0.053487,0.001165,0.006746,0.008713,0.007270,0.033627,0.033762,0.018784,0.007551,0.010386
>2HMQ_Themiste_dyscritum1,0.0,0.000000,0.000000,0.006066,0.005376,0.068845,0.002559,0.040277,0.028729,0.039954,0.014649,0.021153,0.013023,0.014727,0.047270,0.019654,0.004306,0.021575,0.004606
>1FHA_Homo_sapiens1,0.0,0.000000,0.000000,0.000000,0.000892,0.071594,0.008441,0.042979,0.026100,0.042875,0.011943,0.018483,0.010309,0.012022,0.044741,0.022700,0.009044,0.018902,0.005605
>1RCI_Rana_catesbeiana1,0.0,0.000000,0.000000,0.000000,0.000000,0.070824,0.007754,0.042198,0.026899,0.042057,0.012766,0.019294,0.011134,0.012844,0.045509,0.021847,0.008406,0.019713,0.004929
>1PLC_Populus_nigra2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.068398,0.042007,0.093545,0.031528,0.081355,0.086449,0.079843,0.081524,0.109378,0.051832,0.065554,0.087477,0.072840
>4FGF_Homo_sapiens2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039668,0.029361,0.039478,0.015300,0.021795,0.013676,0.015378,0.047879,0.019159,0.003640,0.022212,0.005116
>1NOAStreptomyces_carzinostaticus2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066622,0.022772,0.053714,0.059501,0.052064,0.053748,0.083741,0.028445,0.036442,0.060052,0.044327
>1CDC_Rattus_norvegicus2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066188,0.014522,0.008497,0.016188,0.014421,0.019643,0.046994,0.032792,0.007477,0.024633


In [17]:
dmat_mx['Pdbid']=df['Pdbid']
dmat_mx.set_index('Pdbid', inplace = True)
dmat_mx

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Pdbid,,,,,,,,,,,,,,,,,,,
>1MBA_Aplysia_limacina1,0.0,0.01863,0.007959,0.012388,0.011618,0.028872,0.007964,0.016434,0.024585,0.010430,0.018071,0.021915,0.013423,0.015033,0.043496,0.003471,0.008048,0.018439,0.010342
>1RCB_Homo_sapiens1,0.0,0.00000,0.010682,0.006242,0.007012,0.036851,0.012223,0.024414,0.008610,0.018581,0.000559,0.003285,0.005206,0.003628,0.024866,0.018682,0.014286,0.007321,0.008288
>2HMQ_Themiste_dyscritum1,0.0,0.00000,0.000000,0.004441,0.003670,0.036831,0.001541,0.024393,0.016638,0.018389,0.010123,0.013967,0.007539,0.007801,0.035549,0.008000,0.003603,0.011495,0.002395
>1FHA_Homo_sapiens1,0.0,0.00000,0.000000,0.000000,0.000771,0.036912,0.005982,0.024475,0.014245,0.018471,0.006084,0.009526,0.009001,0.009263,0.031108,0.012441,0.008044,0.012956,0.003542
>1RCI_Rana_catesbeiana1,0.0,0.00000,0.000000,0.000000,0.000000,0.036908,0.005211,0.024471,0.014342,0.018466,0.006453,0.010297,0.009098,0.009360,0.031879,0.011670,0.007274,0.013053,0.003639
>1PLC_Populus_nigra2,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.036836,0.031100,0.036746,0.018441,0.036795,0.036675,0.036711,0.036774,0.043392,0.032342,0.036920,0.036769,0.036877
>4FGF_Homo_sapiens2,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024399,0.018178,0.018394,0.011664,0.015508,0.007017,0.008626,0.037089,0.006459,0.002063,0.012032,0.003935
>1NOAStreptomyces_carzinostaticus2,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024702,0.017104,0.024358,0.024238,0.024274,0.024337,0.043614,0.019905,0.024482,0.024332,0.024440
>1CDC_Rattus_norvegicus2,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024536,0.008162,0.005599,0.011162,0.009552,0.018911,0.024637,0.020241,0.006146,0.014243


In [18]:
dmat=pd.concat([dmat_mh,dmat_ed,dmat_sd,dmat_mx])
dmat.to_csv('PDB_19_Primary_results.csv')